In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import xarray
import cartopy.crs as ccrs
from scipy.signal import detrend
from eofs.xarray import Eof
import iris
from eofs.multivariate.iris import MultivariateEof
import iris.quickplot as qplt
from tempfile import TemporaryFile

In [2]:
variable_mslp = ["PRMSL_GDS0_MSL","mslp"]
variable_rh = ["RH_GDS0_ISBL","rh"]
variable_spfh = ["SPFH_GDS0_ISBL","spfh"]
pfad = "rea/"

In [3]:
def open_df(pfad):
    df = xr.open_mfdataset(pfad+'/*.nc',chunks={'initial_time0_hours':504,'g0_lat_1':29,'g0_lon_2':29})
    df = df.rename({'initial_time0_hours':'time','g0_lat_1':'lat','g0_lon_2':'lon'}).drop("initial_time0_encoded")
    df.coords['lon'] = (df.coords['lon'] + 180) % 360 - 180
    return df
def pref_df(ds_prep,window_size):
        ds_prep = ds_prep.resample(time='1D').mean()
        ds_prep = ds_prep.chunk({'time':-1})
        ds_prep_roll = ds_prep.rolling(time=window_size, center=True).construct('window_dim')
        ds_prep_clim = ds_prep_roll.groupby('time.dayofyear').mean(dim=['window_dim','time'])
        ds_prep_std = ds_prep_roll.groupby('time.dayofyear').std(dim=xr.ALL_DIMS)
        ds_prep = ds_prep.groupby('time.dayofyear') - ds_prep_clim
        ds_prep = ds_prep.groupby('time.dayofyear') / ds_prep_std
        ds_prep = ds_prep.chunk({'time': -1})
        return ds_prep

In [4]:
df = open_df(pfad)
df = df.rename({variable_mslp[0]:variable_mslp[1],variable_rh[0]:variable_rh[1],
                variable_spfh[0]:variable_spfh[1]}).drop("initial_time0")
df = pref_df(df,21)

C:\Users\M\Anaconda3\lib\site-packages\xarray\core\indexing.py:1195: PerformanceWarning: Slicing with an out-of-order index is generating 14 times more chunks
  return self.array[key]


In [5]:
df.to_netcdf('Anomalien.nc')
df

<xarray.Dataset>
Dimensions:    (lat: 29, lon: 29, time: 5113)
Coordinates:
  * lat        (lat) float32 67.5 66.25 65.0 63.75 ... 36.25 35.0 33.75 32.5
  * lon        (lon) float32 -10.0 -8.75 -7.5 -6.25 ... 21.25 22.5 23.75 25.0
  * time       (time) datetime64[ns] 1958-01-01 1958-01-02 ... 1971-12-31
    dayofyear  (time) int64 dask.array<shape=(5113,), chunksize=(5113,)>
Data variables:
    mslp       (time, lat, lon) float32 dask.array<shape=(5113, 29, 29), chunksize=(5113, 29, 29)>
    rh         (time, lat, lon) float32 dask.array<shape=(5113, 29, 29), chunksize=(5113, 29, 29)>
    spfh       (time, lat, lon) float32 dask.array<shape=(5113, 29, 29), chunksize=(5113, 29, 29)>

In [6]:
df_roll = df.rolling(time=21,center=True).construct("window_dim")

In [7]:
vars_list = []
eofs_list = []
pcs_list = []
evs_list = []
solvers_list = []
i_list = []

l = 0

for day in df_roll.groupby("time.dayofyear"):
    cube = day[1].rename({"time":
                          "time_old"}).stack(time = ("time_old",
                                                            "window_dim")).reset_index("time").transpose("time",
                                                                                                         "lat",
                                                                                                         "lon").dropna("time")
    cube_mslp = cube.mslp.assign_coords(time=range(0,len(cube.time))).to_iris()
    cube_rh = cube.rh.assign_coords(time=range(0,len(cube.time))).to_iris()
    cube_spfh = cube.spfh.assign_coords(time=range(0,len(cube.time))).to_iris()
    solver = MultivariateEof([cube_mslp,cube_rh,cube_spfh],weights="coslat")
    
    print(l)
    #print(day[1].time)
    
    var = 0
    i = 1
    var_list = []
    while var < 0.9:
        v = solver.varianceFraction(neigs=i)
        var = v.data.sum()
        var_list += [var]
        i += 1
    
    vars_list += [var_list]
    eofs_list += [xr.DataArray.from_iris(solver.eofs(neofs=i)[0]).rename('eofs')]
    pcs_list += [xr.DataArray.from_iris(solver.pcs(npcs=i)).rename('pcs')]
    evs_list += [xr.DataArray.from_iris(solver.eigenvalues(neigs=i)).rename('evs')]
    solvers_list += [solver]
    i_list += [i]
    
    l += 1
    
    #if l == 3:
    #    break

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25


KeyboardInterrupt: 

In [ ]:
#i_list

In [ ]:
#eofs_list

In [ ]:
#pcs_list

In [ ]:
#evs_list

In [ ]:
#vars_list

In [ ]:
#solvers_list

In [9]:
#EOFs_Analyse.seek(0)
#EOFs_Analyse = TemporaryFile()
np.savez("EOFs_Analyse.npz",
         eofs=eofs_list,
         neofs=i_list,
         pcs=pcs_list,
         evs=evs_list,
         var=vars_list,
         solvers=solvers_list)

In [ ]:
#plt.axes(projection=ccrs.PlateCarree())
#qplt.contourf(eofs_list[0][0])
#plt.gca().coastlines()
#plt.show()